#### Define table schemas 

In [125]:
import os
import sys
import pandas as pd
import logging
from typing import List
from google.cloud import bigquery 
from google.oauth2 import service_account

key_path = "/home/alex/.creds/salex-sa.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

DATA_DIR = "../data"
PROJECT_NAME = "team-week2"

# **** TABLE SCHEMAS ****
TABLE_METADATA = {
   ## 'Raw' Data (just renaming columns)
   'chi_crimes': {
       'dataset_name':'chicago',
       'table_name': 'chi_crimes',
       'schema': [
           bigquery.SchemaField('crime_id', 'INTEGER', mode='REQUIRED'),
           bigquery.SchemaField('case_number', 'STRING', mode='REQUIRED'),
           bigquery.SchemaField('date_time', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('block', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('IUCR_code', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('primary_IUCR', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('secondary_IUCR', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('location_description', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('arrest', 'BOOL', mode='NULLABLE'),
           bigquery.SchemaField('domestic', 'BOOL', mode='NULLABLE'),
           bigquery.SchemaField('beat', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('district', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('ward', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('community_area', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('FBI_code', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('x_coordinate', 'FLOAT', mode='NULLABLE'),
           bigquery.SchemaField('y_coordinate', 'FLOAT', mode='NULLABLE'),
           bigquery.SchemaField('year', 'INTEGER', mode='NULLABLE'),
           bigquery.SchemaField('updated_on', 'STRING', mode='NULLABLE'),
           bigquery.SchemaField('lat', 'FLOAT', mode='NULLABLE'),
           bigquery.SchemaField('lon', 'FLOAT', mode='NULLABLE'), 
           bigquery.SchemaField('location', 'STRING', mode='NULLABLE'),
       ]
   }, 'den_crimes': {
        'dataset_name':'denver',
        'table_name':'den_crimes',
        'schema': [
            bigquery.SchemaField('incident_id', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_id', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_code', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('offense_code_ext', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('offense_type_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_cat_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('first_occurrence', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('last_occurrence', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('reported_date', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('address', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('geo_x', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('geo_y', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('geo_lon', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('geo_lat', 'FLOAT', mode='NULLABLE'),
            bigquery.SchemaField('district_id', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('precinct_id', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('neighborhood_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('is_crime', 'BOOL', mode='NULLABLE'), 
            bigquery.SchemaField('is_traffic', 'BOOL', mode='NULLABLE'), 
            bigquery.SchemaField('victim_count', 'INTEGER', mode='NULLABLE')
          ] 
   }, 'off_codes':{
        'dataset_name':'denver',
        'table_name':'off_codes',
        'schema': [
            bigquery.SchemaField('object_id', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_code', 'INTEGER', mode='REQUIRED'),
            bigquery.SchemaField('offense_code_ext', 'INTEGER', mode='NULLABLE'),
            bigquery.SchemaField('offense_type_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_type_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_cat_id', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('offense_cat_name', 'STRING', mode='NULLABLE'),
            bigquery.SchemaField('is_crime', 'BOOL', mode='NULLABLE'),
            bigquery.SchemaField('is_traffic', 'BOOL', mode='NULLABLE')

        ]
   }
}

# **** BIGQUERY CLIENT ****
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# **** CREATE TABLES  ****
for table_name in TABLE_METADATA:
  # Create dataset (if needed)
  dataset_name = TABLE_METADATA[table_name]['dataset_name']
  dataset_id = f"{PROJECT_NAME}.{dataset_name}"
  dataset = bigquery.Dataset(dataset_id)
  dataset.location = "US"
  dataset = client.create_dataset(dataset, exists_ok=True)
  # Create table
  full_table_id = f"{PROJECT_NAME}.{dataset_name}.{table_name}"
  schema = TABLE_METADATA[table_name]['schema']
  table = bigquery.Table(full_table_id, schema = schema)
  try:
    client.create_table(table)
    print(f"Created table: {table_name}")
    table_ref = client.get_table(table)
    for column in table_ref.schema:
      print(f"\t{column.name}\t{column.field_type}") 
    print("\n")
  except:
    print(f"Did not create table {table_name}. Already exists?")

Did not create table chi_crimes. Already exists?
Created table: den_crimes
	incident_id	INTEGER
	offense_id	INTEGER
	offense_code	INTEGER
	offense_code_ext	INTEGER
	offense_type_id	STRING
	offense_cat_id	STRING
	first_occurrence	DATETIME
	last_occurrence	DATETIME
	reported_date	DATETIME
	address	STRING
	geo_x	FLOAT
	geo_y	FLOAT
	geo_lon	FLOAT
	geo_lat	FLOAT
	district_id	INTEGER
	precinct_id	INTEGER
	neighborhood_id	STRING
	is_crime	BOOLEAN
	is_traffic	BOOLEAN
	victim_count	INTEGER


Did not create table off_codes. Already exists?


In [115]:
# job_config = bigquery.LoadJobConfig(
        # create_disposition="CREATE_IF_NEEDED",
        # write_disposition="WRITE_APPEND",
        # schema=TABLE_METADATA['chi_crimes']['schema']
#     )
# 
# for year in range(18,23):
#   filepath = f"{DATA_DIR}/chicago/20{year}.csv"
#   print(filepath)
#   df = pd.read_csv(filepath)
#   df.columns = [col.lower().replace(" ","_") for col in df.columns]
#   df.rename({'id':'crime_id', 
        #   'date':'date_time',
        #   'iucr':'IUCR_code',
        #   'primary_type':'primary_IUCR',
        #   'description':'secondary_IUCR',
        #   'fbi_code':'FBI_code',
        #   'latitude':'lat',
        #   'longitude':'lon'
        #   }, axis=1, inplace=True)
#   job = client.load_table_from_dataframe(df, destination=f'{PROJECT_NAME}.chicago.chi_crimes', job_config=job_config)
#   job.result()

../data/chicago/2018.csv
../data/chicago/2019.csv
../data/chicago/2020.csv
../data/chicago/2021.csv
../data/chicago/2022.csv


In [129]:
with open("../data/denver/crime.csv", "r", encoding="windows-1252") as file:
  den_crimes = pd.read_csv(file, index_col=0)

In [132]:
den_crimes.columns = [col.lower().replace("e_date","e") for col in den_crimes.columns]
den_crimes.rename({
  'offense_code_extension':'offense_code_ext',
  'offense_category_id':'offense_cat_id',
  'incident_address':'address'
}, axis=1, inplace=True)
den_crimes.head(1)

,offense_id,offense_code,offense_code_ext,offense_type_id,offense_cat_id,first_occurrence,last_occurrence,reported_date,address,geo_x,geo_y,geo_lon,geo_lat,district_id,precinct_id,neighborhood_id,is_crime,is_traffic,victim_count
incident_id,,,,,,,,,,,,,,,,,,,
2017421909,2017421909299900,2999,0,criminal-mischief-other,public-disorder,6/25/2017 8:40:00 PM,NaN,6/27/2017 7:01:00 PM,2920 W 32ND AVE,3133773.0,1702660.0,-105.024167,39.761646,1,113,highland,1,0,1


In [ ]:
den_crimes.columns = [col.lower() for col in den_crimes.columns]



In [116]:
job_config = bigquery.LoadJobConfig(
        create_disposition="CREATE_IF_NEEDED",
        write_disposition="WRITE_TRUNCATE",
        schema=TABLE_METADATA['den_crimes']['schema']
    )
df = pd.read_csv("../data/denver/crime.csv")
df.head(1)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 78792: invalid start byte

#### Create/load to staging table for chicago crime data  

In [53]:
# def loadStagingTable(file_name): 
#   dataset_name = "chicago"
#   staging_table_name = "tmp_chi_crime"
#   full_table_id = f"{PROJECT_NAME}.{dataset_name}.{staging_table_name}"

#   df = pd.read_csv(file_name)
#   df[['created_at']] = None
#   df[['modified_at']] = None

#   job_config = bigquery.LoadJobConfig(
#     schema=TABLE_METADATA['chi_crimes']['schema'], 
#     create_disposition="CREATE_IF_NEEDED",
#     write_disposition="WRITE_TRUNCATE",
#     destination_table_description="Staging table for loading chicago crime data",
#   )

#   job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
#   job.result()
#   print(f"loaded {job.output_rows} rows into {job.destination}")

#   query = f"""
# MERGE INTO team-week2.chicago.chi_crimes as trg
# USING team-week2.chicago.tmp_chi_crime as src
# ON
#   trg.license_plate = src.license_plate
# WHEN MATCHED THEN 
#   UPDATE SET year = src.year,
#   make = src.make,
#   model = src.model,
#   first_name = src.first_name,
#   last_name = src.last_name,
#   birth_date = src.birth_date,
#   street_address = src.street_address,
#   city = src.city,
#   state = src.state,
#   zip = src.zip, 
#   modified_at = CURRENT_TIMESTAMP
# WHEN NOT MATCHED THEN 
# INSERT (
#     license_plate,
#     vin,
#     year,
#     make,
#     model,
#     first_name,
#     last_name,
#     birth_date,
#     street_address,
#     city,
#     state,
#     zip, 
#     created_at
# ) VALUES (
#     src.license_plate,
#     src.vin,
#     src.year,
#     src.make,
#     src.model,
#     src.first_name,
#     src.last_name,
#     src.birth_date,
#     src.street_address,
#     src.city,
#     src.state,
#     src.zip, 
#     CURRENT_TIMESTAMP
#     ); 
# """
# result = client.query(query)


399572